In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [2]:
# для google colab
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# розмір картинки
IMG_SIZE = (75, 75)

# папка з датасетом у якій підпапки з фотографіями кожного символа 
# назви класів будуть відповідати іменам підпапок
directory = "images/"

In [4]:
data_generator = ImageDataGenerator(validation_split = 0.2, rescale = 1.0 / 255)
 
# датасет для тренування
train = data_generator.flow_from_directory(directory,
                                               target_size = IMG_SIZE,
                                               class_mode = 'categorical',
                                               subset = 'training')

# датасет для тестування
test = data_generator.flow_from_directory(directory,
                                               target_size = IMG_SIZE,
                                               class_mode = 'categorical',
                                               subset = 'validation')

Found 286 images belonging to 29 classes.
Found 55 images belonging to 29 classes.


In [5]:
def createModel(image_shape = IMG_SIZE):
    '''
    Аргументи:
        image_shape -- ширина та висота картинки
    Повертає:
        tf.keras.model
    '''
    input_shape = image_shape + (3,)
    
    base_model = InceptionV3(input_shape = input_shape,
                            include_top = False,
                            weights = "imagenet",
                            classes = 29,
                            classifier_activation = "softmax")
    # заморозимо шари, щоб їх не тренувати
    base_model.trainable = False

    # вхідний шар
    inputs = tf.keras.Input(shape = input_shape) 

    # для нормалізації інпута [-1;1]
    x = tf.keras.applications.inception_v3.preprocess_input(inputs) 

    x = base_model(x, training = False) 
    x = tfl.GlobalAveragePooling2D()(x) 
    x = tfl.Dropout(0.2)(x)
    outputs = tfl.Dense(29, activation = "softmax")(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [6]:
model = createModel(IMG_SIZE)

2022-12-06 13:22:44.352027: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-06 13:22:44.352974: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


In [7]:
# метрики
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision_f1 = precision(y_true, y_pred)
    recall_f1 = recall(y_true, y_pred)
    return 2 * ((precision_f1 * recall_f1) / (precision_f1 + recall_f1 + K.epsilon()))

In [8]:
base_learning_rate = 0.01

initial_epochs = 10

loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits = True)
optimizer = tf.keras.optimizers.Adam(base_learning_rate)
metrics = ['acc', f1_score, precision, recall]

In [9]:
model.compile(optimizer = optimizer,
              loss = loss_function,
              metrics = metrics)

In [10]:
history = model.fit(train,
                    validation_data = test,
                    epochs = initial_epochs)

Epoch 1/10


2022-12-06 13:22:51.625825: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
/Users/tylerdurden/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/backend.py:5531: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2022-12-06 13:22:52.821930: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


8/9 [=========================>....] - ETA: 0s - loss: 3.7226 - acc: 0.3780 - f1_score: 0.3745 - precision: 0.5060 - recall: 0.3010             

2022-12-06 13:22:59.052847: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 8s 283ms/step - loss: 3.5041 - acc: 0.4056 - f1_score: 0.4084 - precision: 0.5450 - recall: 0.3301 - val_loss: 0.7491 - val_acc: 0.8364 - val_f1_score: 0.8113 - val_precision: 0.9244 - val_recall: 0.7228
Epoch 2/10
9/9 [==============================] - 1s 61ms/step - loss: 1.2882 - acc: 0.7308 - f1_score: 0.7501 - precision: 0.8256 - recall: 0.6900 - val_loss: 0.3016 - val_acc: 0.9091 - val_f1_score: 0.9217 - val_precision: 0.9516 - val_recall: 0.8940
Epoch 3/10
9/9 [==============================] - 1s 56ms/step - loss: 0.6242 - acc: 0.8881 - f1_score: 0.8815 - precision: 0.9215 - recall: 0.8465 - val_loss: 0.1941 - val_acc: 0.9091 - val_f1_score: 0.9183 - val_precision: 0.9439 - val_recall: 0.8940
Epoch 4/10
9/9 [==============================] - 1s 58ms/step - loss: 0.3899 - acc: 0.9126 - f1_score: 0.9112 - precision: 0.9436 - recall: 0.8815 - val_loss: 0.1896 - val_acc: 0.9818 - val_f1_score: 0.9762 - val_precision: 0.9839 - val_recall: 0.968

In [11]:
# model.summary()

# print("Кількість шарів у базовій моделі: ", len(tunedModel.layers))

In [12]:
tunedModel = model.layers[3]
tunedModel.trainable = True

fine_tune_start = 250

In [13]:
# Заморозимо всі шари перед fine_tune_start
for layer in tunedModel.layers[:fine_tune_start]:
    layer.trainable = False

optimizer = Adam(learning_rate = 0.0001 * base_learning_rate)

fine_tune_epochs = 100
total_epochs =  initial_epochs + fine_tune_epochs

In [14]:
model.compile(optimizer = optimizer,
              loss = loss_function,
              metrics = metrics)

In [15]:
history_fine = model.fit(train,
                         validation_data = test,
                         epochs = total_epochs,
                         initial_epoch = history.epoch[-1])

Epoch 10/110


2022-12-06 13:23:47.004837: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - ETA: 0s - loss: 0.0569 - acc: 0.9825 - f1_score: 0.9791 - precision: 0.9826 - recall: 0.9757

2022-12-06 13:23:51.331350: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 7s 301ms/step - loss: 0.0569 - acc: 0.9825 - f1_score: 0.9791 - precision: 0.9826 - recall: 0.9757 - val_loss: 0.1347 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 11/110
9/9 [==============================] - 1s 95ms/step - loss: 0.0524 - acc: 0.9895 - f1_score: 0.9913 - precision: 0.9931 - recall: 0.9896 - val_loss: 0.1348 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 12/110
9/9 [==============================] - 1s 92ms/step - loss: 0.0457 - acc: 0.9930 - f1_score: 0.9948 - precision: 0.9965 - recall: 0.9931 - val_loss: 0.1361 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 13/110
9/9 [==============================] - 1s 84ms/step - loss: 0.0418 - acc: 0.9930 - f1_score: 0.9913 - precision: 0.9931 - recall: 0.9896 - val_loss: 0.1360 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall:

9/9 [==============================] - 1s 84ms/step - loss: 0.0253 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1504 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 43/110
9/9 [==============================] - 1s 86ms/step - loss: 0.0208 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1502 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 44/110
9/9 [==============================] - 1s 84ms/step - loss: 0.0267 - acc: 0.9930 - f1_score: 0.9931 - precision: 0.9931 - recall: 0.9931 - val_loss: 0.1505 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 45/110
9/9 [==============================] - 1s 84ms/step - loss: 0.0303 - acc: 0.9930 - f1_score: 0.9931 - precision: 0.9931 - recall: 0.9931 - val_loss: 0.1508 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 

9/9 [==============================] - 1s 83ms/step - loss: 0.0168 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1611 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 75/110
9/9 [==============================] - 1s 83ms/step - loss: 0.0232 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1588 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 76/110
9/9 [==============================] - 1s 89ms/step - loss: 0.0178 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1600 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 77/110
9/9 [==============================] - 1s 84ms/step - loss: 0.0196 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1598 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 

9/9 [==============================] - 1s 84ms/step - loss: 0.0176 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1624 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 107/110
9/9 [==============================] - 1s 85ms/step - loss: 0.0142 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1621 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 108/110
9/9 [==============================] - 1s 84ms/step - loss: 0.0152 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1631 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 109/110
9/9 [==============================] - 1s 94ms/step - loss: 0.0163 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1612 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recal

In [ ]:
model.save('Fine_tuned_model01.h5', overwrite = True, include_optimizer = True)

In [22]:
from PIL import Image
image1 = Image.open("img.png")
data = np.asarray(image1)
model.predict(data)

ValueError: in user code:

    File "/Users/tylerdurden/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "/Users/tylerdurden/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/tylerdurden/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/tylerdurden/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1983, in predict_step
        return self(x, training=False)
    File "/Users/tylerdurden/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) fro